## Documentation

## Package Imports

In [2]:
!pip install simple_salesforce==1.11.3 # Run if first time using code

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
import csv
import pandas as pd
import requests

from simple_salesforce import Salesforce, format_soql, bulk # See Docu for more info about simple-salesforce
sf = Salesforce() # Whitelist your IP on Salesforce if you do not have the security token (See Docu)

## Exporting all Opportunities to CSV — Don't run if not needed (~8 mins runtime)

In [ ]:
# If need to export csv of all Opportunities info on Salesforce 
opportunities_dict = sf.query_all("SELECT Id, Name FROM Opportunity")
opportunities_data = pd.DataFrame(opportunities_dict['records'])

opp_list = []
for index, row in opportunities_data.iterrows():
    opp = sf.Opportunity.get(row['Id'])
    opp_list.append(opp)

opp_df = pd.DataFrame(opp_list)
opp_df.head()
opp_df.to_csv('all_opportunities_data_aug8.csv') # Will download on 'Files' section of Deepnote

## Importing info to Opportunities

In [4]:
# Please don't forget to run this cell :)
opportunities_dict = sf.query_all("SELECT Id, Name FROM Opportunity") 
opportunities_data = pd.DataFrame(opportunities_dict['records'])

In [ ]:
opportunities_data.tail()

In [ ]:
# IMPORT CSV WITH DATA YOU WANT TO IMPORT
import_df = pd.read_csv("Post Interview - For Upload.csv") # Filename of CSV you uploaded on 'Files' section. Don't forget '.csv'!
import_df = import_df.fillna('')
#import_df.head(2)
import_df

In [6]:
#We make a list of all the fields from the csv file
import_df_cols = list(import_df.columns.values)

In [ ]:
# Bulk Updating on Salesforce
import_data = []
no_match_data = []

for index, row in import_df.iterrows():
    import_dict = {}
    name = row['Name']
    
    if name in opportunities_data.values:
        opportunity_id = opportunities_data[opportunities_data['Name']==name].iloc[0]['Id']
        import_dict['Id'] = opportunity_id
        
        # FORMAT: import_dict['Salesforce Field Name'] = row['Name of Data Column on your CSV']
        # EXAMPLE: import_dict['Address__c'] = row['Address'] 
        #import_dict['Address__c'] = row['Address__c'] 
        # To find list of Salesforce Field Names, see Documentation.
        #import_dict['Date_of_Team_Review__c'] = row['Date_of_Team_Review__c']
        #we will go over every field in our csv and match agaisnt sf fields
        for col in import_df_cols:
            import_dict[col] = row[col]
        
        import_data.append(import_dict)
    else:
        no_match_data.append(name)

print(import_data) # Prints list of all data that will get imported.
print(no_match_data) # Prints list of opportunities not found by name on Salesforce. Might be due to typos.
sf.bulk.Opportunity.update(import_data, batch_size=1000,use_serial=True)
#sf.bulk.Opportunity.update(import_data,use_serial=True)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2ff65363-a8b1-4ea1-87ff-885722518a14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>